In [1]:
import torch
from torch import nn, optim, autograd as grad
from torch.utils.data import DataLoader, random_split, dataset
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from AddClass import *

In [2]:
Embed_dim = 300
word_num = 10_000
num_layers = 2
batch_size = 64

In [3]:
class Rnn(nn.Module):
    
    def __init__(self):
        super(Rnn, self).__init__()
        self.rnnL1 = nn.RNN(input_size=Embed_dim ,hidden_size=1024,
                nonlinearity= 'tanh' ,num_layers=num_layers, batch_first= True)
        self.linL2 = nn.Linear(1024,5)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
    def forward(self,x):
        x,hidden =self.rnnL1(x)
        x=self.relu(
            self.linL2(x)
        )
        x=self.softmax(x)
        
        return(x)
        

Dataset call section

In [4]:
dir_path = r"C:\Users\orian\OneDrive\שולחן העבודה\Coding Files 2\Datasets"
dataset = pd.read_csv(dir_path+r"\chat gpt reviews\ChatGPT_Reviews.csv")
emb_dict = pd.read_csv(dir_path+r"\embedding dictionary\dict(2).csv")

print(emb_dict.keys())

# Create a dataset dictionary of reviews and ratings
part_ds = [dataset['Review'], torch.tensor(dataset['Ratings'])]

temp_part_ds = []
# 
part_ds = list(zip(part_ds[0], part_ds[1]-1))

train_len = round(len(part_ds)*0.9)
train_ds_csv, test_ds_csv = random_split(
    dataset=part_ds,lengths=[train_len, len(part_ds)-train_len])
train_data = DataLoader(train_ds_csv, shuffle= True)

Index(['word'], dtype='object')


Embedding Section

In [5]:
word_dict = emb_dict['word'].tolist()
word_dict = set(word_dict)

# Separate the words in the reviews and make sure they don't repeat themselves
# (They are going to be added to the embeddings)
review_set = {word for sentence in dataset['Review'].astype(str)
               for word in sentence.split(' ')}

combined_words = word_dict.union(review_set)

# Dictionary to map words to indices
word_to_index = {key: val for val, key in enumerate(combined_words)}
embedding_layer = nn.Embedding(num_embeddings=len(word_to_index)
                               , embedding_dim=Embed_dim)

# Example of how to use the embedding layer that needs to be 
# deleted and moved to the feedforward section
input_words = ["suck","the","coconut","juice","out","of","the","coconut"]

# Made to check if the embedding layer works for 
# the training dataset here - it does. See if it gets distorted further down the line
tmp_random_input = list(train_data)[0][0]

# Strip the mentioned values out of each word in the embedded sentence and then turn them into a word array
tmp_random_input = tupleToArray(tmp_random_input)


#input_indices = torch.LongTensor([word_to_index[word] for word in input_words])  # Convert words to indices
input_indices = torch.LongTensor([word_to_index[word] for word in tmp_random_input])  # Convert words to indices
embeddings = embedding_layer(input_indices)
print(embeddings.shape)


torch.Size([3, 300])


Single Epoch Training:

In [6]:
def train_one_epoch(rnn, train_ds, optimizer, dev, FILE_PATH, epoch_iter
                 # Should add stuff that's required
):
    iter = 0

    # Training loop part
    for sample in train_ds:
        rnn = rnn.to(dev) # Added due to errors (Should remain in the loop)
        print(sample[0])
        targetY = torch.nn.functional.one_hot(
            torch.tensor(sample[1]), num_classes=5)
        targetY = torch.tensor(targetY, dtype=torch.float32, requires_grad=True)
        targetY = torch.squeeze(targetY).to(dev)
        
        # ---ForwardPropagation---
        input_words = tupleToArray(sample[0])
        embed_var = torch.LongTensor([word_to_index[word] for word in input_words])
        embed_var = embedding_layer(embed_var).to(dev)
        print(embed_var.shape)
        print(embed_var)
        pred = rnn.forward(embed_var).to(dev)
        loss_func = nn.CrossEntropyLoss()

        # ---BackPropagation---
        loss = loss_func(pred, targetY)
        """ Use this link for the error:
         https://discuss.pytorch.org/t/valueerror-expected-input-batch-size-324-to-match-target-batch-size-4/24498/4
        """
        loss.backward()
        optimizer.step()

        if iter%100==0:
            torch.save(rnn.cpu(), FILE_PATH)
            grad_tmp = rnn.linL2.weight.grad
            print(iter,"({})".format(epoch_iter+1), " | ",grad_tmp.sum()**2/len(grad_tmp))

        optimizer.zero_grad()
        iter+=1
    

Main Section

In [7]:
if __name__ == "__main__":
    if torch.cuda.is_available() : device = "cuda:0"
    else : device = "cpu"
    device = torch.device(device)
        
    rnn = Rnn().to(device)
    train_one_epoch(rnn, train_data, optimizer=optim.Adam(rnn.parameters(), lr=0.001), dev=device, FILE_PATH="model.pth", epoch_iter=0)
    

('Wonderfull App 🫀',)


C:\Users\orian\AppData\Local\Temp\ipykernel_15380\464720283.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(sample[1]), num_classes=5)
C:\Users\orian\AppData\Local\Temp\ipykernel_15380\464720283.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targetY = torch.tensor(targetY, dtype=torch.float32, requires_grad=True)


torch.Size([3, 300])
tensor([[ 7.5703e-01,  4.6492e-01,  4.3981e-01, -1.3652e+00, -1.2562e+00,
         -2.3900e-01, -1.7695e+00,  8.9601e-01, -3.4862e-01,  1.0219e+00,
         -5.2816e-01, -5.0088e-01,  2.5484e+00, -2.3730e-01,  1.0673e+00,
         -1.0533e-02, -4.4679e-01,  7.6384e-02, -9.3777e-02, -7.3052e-01,
         -4.5731e-01,  1.2461e+00, -3.0539e-01, -1.1649e+00, -6.7791e-01,
         -5.0765e-01,  5.6716e-01,  5.1103e-01,  5.1356e-01, -7.7214e-01,
         -1.4037e-01,  3.7745e-02, -4.1944e-01, -1.0599e+00,  6.4527e-01,
          1.7221e+00, -2.7483e-01, -1.9844e-01, -1.2447e+00,  5.8512e-01,
          1.7336e-01,  1.4955e+00, -9.7597e-01, -2.3586e-01, -4.9877e-01,
         -1.1311e+00, -1.2403e+00,  2.1800e-01,  9.9735e-01, -8.1162e-01,
          1.4817e-01, -8.3533e-01, -5.8241e-01,  8.7149e-01,  8.6926e-01,
         -1.5881e+00,  1.2443e-02, -2.4655e-01,  1.8479e-01, -6.7236e-01,
          8.8967e-01,  3.6975e-01,  2.9754e-02, -9.1278e-02, -5.7050e-01,
          1.0228e

c:\Users\orian\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


ValueError: Expected input batch_size (3) to match target batch_size (5).